In [10]:
from policies import Trajectory, p_s_from_rollouts, p_sa_from_rollouts, sr_from_rollouts, average_reward_from_rollouts
from plotting import plot_heatmap
import aggregation
import pickle

def read_pickle(save_name):
    """
    Thread-safe pickle read.
    Safe even if other threads are simultaneously calling dump_pickle()
    on the same file.
    """
    with open(save_name, "rb") as f:
        return pickle.load(f)

In [18]:

runs = read_pickle("out/mountaincar/eigenoptions/all_l1_covs.pkl")

runs_codex = read_pickle("out/mountaincar/codex/all_l1_covs.pkl")


In [21]:
print(len(runs))
print(runs)
print(runs_codex)

104
[[np.float64(37.363470509909206), np.float64(4.09337079084475), np.float64(2.56503384599102)], [np.float64(34.13919492733445), np.float64(8.391178683912965), np.float64(4.191352679757124)], [np.float64(36.65445844393268), np.float64(5.822408742090797), np.float64(3.617805814765939)], [np.float64(33.51077037563405), np.float64(5.6159451860189895), np.float64(3.419701357372824)], [np.float64(34.57735667761531), np.float64(3.6724950919546693), np.float64(2.7351789212170434)], [np.float64(35.82920957039818), np.float64(4.318069712105255), np.float64(2.6752320796006783)], [np.float64(37.16491082047679), np.float64(3.964793727587698), np.float64(3.050445271394926)], [np.float64(32.74189322260503), np.float64(4.962358840318963), np.float64(3.818247749057209)], [np.float64(31.37766945895332), np.float64(7.0529140249855615), np.float64(3.8199037311930955)], [np.float64(35.1854786649321), np.float64(3.864793462119674), np.float64(2.735223009304962)], [np.float64(36.05127575209046), np.float6

TypeError: list indices must be integers or slices, not str